<a href="https://colab.research.google.com/github/PriyaMurali/Data-Analysis/blob/main/HiveMySQLDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from collections import Counter
!wget -q https://dlcdn.apache.org/spark/spark-3.3.0/spark-3.3.0-bin-hadoop3.tgz
!tar xf spark-3.3.0-bin-hadoop3.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

In [2]:
!apt-get update
!apt-get install --fix-missing openjdk-8-jdk-headless -qq > /dev/null


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [90.7 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [903 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports In

In [3]:
!pip install mysql-connector-python


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 25.4 MB 60.8 MB/s 


In [4]:
import findspark
findspark.init()
from pyspark.sql import SparkSession

In [ ]:
#HIve Mysql Example
import mysql.connector
import pandas as pd

appName = "PySpark MySQL Example - via mysql.connector"
master = "local"

spark = SparkSession.builder.master(master).appName(appName).getOrCreate()

# Establish a connection
conn = mysql.connector.connect(user='***', database='***',
                               password='***',
                               host="localhost",
                               port=10101)
cursor = conn.cursor()
query = "SELECT id, value FROM test_table"
# Create a pandas dataframe
pdf = pd.read_sql(query, con=conn)
conn.close()

# Convert Pandas dataframe to spark DataFrame
df = spark.createDataFrame(pdf)

df.show()

In [ ]:
from pyspark.sql import HiveContext
hc = HiveContext(spark.sparkContext)
print(hc)


/content/spark-3.3.0-bin-hadoop3/python/pyspark/sql/context.py:721: FutureWarning: HiveContext is deprecated in Spark 2.0.0. Please use SparkSession.builder.enableHiveSupport().getOrCreate() instead.
  FutureWarning,
/content/spark-3.3.0-bin-hadoop3/python/pyspark/sql/context.py:114: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning,


In [ ]:
hc.sql('create database db') #where db is the database name
hc.sql('use db')

DataFrame[]

In [ ]:
##Storing Data in HiveContext
from pyspark.sql.types import StructType, StructField, FloatType, StringType
iris_schema = StructType([
StructField("Sepal_Length", FloatType(), True),
StructField("Sepal_Width", FloatType(), True),
StructField("Petal_Length", FloatType(), True),
StructField("Petal_Width", FloatType(), True),
StructField("Species", StringType(), True)
])
iris1_df1 = spark.read.csv(path='/content/iris.csv',sep=',',header=True,schema=iris_schema)
hc.sql('use db')
from pyspark.sql import DataFrameWriter
dfw = DataFrameWriter(iris1_df1)
dfw.saveAsTable(name="iris",mode='overwrite')


In [ ]:
hc.tableNames()

['iris']

In [ ]:
#Retriving Data from HiveContext
hc.sql('Select count(*) as RecordCount from iris').show()

+-----------+
|RecordCount|
+-----------+
|        150|
+-----------+



In [5]:
#SQLSESSION
spark = SparkSession \
        .builder \
        .appName("PySparkSQLTelecom") \
        .getOrCreate()
sc = spark.sparkContext

In [9]:
Customer = sc.textFile("/content/LexSparkCompanyInfoExample.csv")

In [16]:
splitCustomer = Customer.map(lambda var1: var1.split(","))

In [23]:
columns=["CompanyID","CompanyName","CompanyLocation","CompanyAddress","CompanyContact","ProfitMargin","EstablishedYear"]
dfCompanyInfo = spark.createDataFrame(splitCustomer,columns)
dfCompanyInfo.show()

+---------+----------------+---------------+--------------------+--------------+------------+---------------+
|CompanyID|     CompanyName|CompanyLocation|      CompanyAddress|CompanyContact|ProfitMargin|EstablishedYear|
+---------+----------------+---------------+--------------------+--------------+------------+---------------+
|    C1001|    Steel Wheels|   Pennsylvania|    80 Stang Parkway|    7172809057|  4061361.08|           2007|
|    C1002|   MotoLab Bikes|          Texas|     1 Anthes Avenue|    6829309477|  2733417.09|           2011|
|    C1003|        Lee Auto|     California|   87985 Linden Pass|    6193601290|  4422985.26|           1994|
|    C1004|     Zoom Dynamo|          Texas| 56 La Follette Pass|    8327309957|  3102258.84|           1993|
|    C1005|Quakesons Motors|          Texas|  8 Briar Crest Pass|    8175902345|  3584342.27|           2002|
|    C1006|    Steel Wheels|   Pennsylvania|    035 Iowa Terrace|    5706889534|  3169758.24|           2009|
|    C1007

In [34]:
from pyspark.sql.functions import col
dfCompanyInfo = dfCompanyInfo.withColumn("CompanyContact",col("CompanyContact").cast(IntegerType()))\
                .withColumn("ProfitMargin",col("ProfitMargin").cast(DoubleType()))\
                .withColumn("EstablishedYear",col("EstablishedYear").cast(IntegerType()))
dfCompanyInfo.dtypes


[('CompanyID', 'string'),
 ('CompanyName', 'string'),
 ('CompanyLocation', 'string'),
 ('CompanyAddress', 'string'),
 ('CompanyContact', 'int'),
 ('ProfitMargin', 'double'),
 ('EstablishedYear', 'int')]

In [31]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType ,DoubleType
schemaList = StructType() \
      .add("OrderID",StringType(),True) \
      .add("CompanyID",StringType(),True) \
      .add("CompanyName",StringType(),True) \
      .add("CompanyLocation",StringType(),True) \
      .add("ComponentToBeManufactured",StringType(),True) \
      .add("Quantity",IntegerType(),True) \
      .add("EstimatedCost",DoubleType(),True) \
      .add("OrderDate",StringType(),True) \
      .add("DueDate",StringType(),True) \
      .add("CompletionStatus",StringType(),True) \
      .add("DeliveryStatus",StringType(),True) \

In [32]:
#headers =["OrderID","CompanyID","CompanyName","CompanyLocation","ComponentToBeManufactured","Quantity","EstimatedCost","OrderDate","DueDate","CompletionStatus","DeliveryStatus"]
dfCompanyOrders = spark.read.csv("/content/LexSparkCompanyOrdersExample.csv",schema=schemaList) 
dfCompanyOrders.show()

+-------+---------+----------------+---------------+-------------------------+--------+-------------+---------+---------+----------------+--------------+
|OrderID|CompanyID|     CompanyName|CompanyLocation|ComponentToBeManufactured|Quantity|EstimatedCost|OrderDate|  DueDate|CompletionStatus|DeliveryStatus|
+-------+---------+----------------+---------------+-------------------------+--------+-------------+---------+---------+----------------+--------------+
|OD10001|    C1108|     Raido Bikes|        Georgia|             Spring Forks|   21103|   9487867.48| 3-Nov-19|24-Dec-19|         Pending|       Pending|
|OD10002|    C1035| Ilmio Jane Moto| North Carolina|                   Fender|   26272|   4899369.32|18-Dec-19|14-Mar-20|         Pending|       Pending|
|OD10003|    C1174|   MotoLab Bikes|         Nevada|                Generator|   39789|   4008848.49| 7-Dec-19|31-Jan-20|       Completed|     Delivered|
|OD10004|    C1121|        Lee Auto|         Oregon|                   Brake

In [35]:
dfCompanyInfo.createOrReplaceTempView("CompanyInfo")
dfCompanyOrders.createOrReplaceTempView("CompanyOrders")


In [36]:
spark.sql("show tables").show()


+---------+-------------+-----------+
|namespace|    tableName|isTemporary|
+---------+-------------+-----------+
|         |  companyinfo|       true|
|         |companyorders|       true|
+---------+-------------+-----------+



In [44]:
#Fetch the details of all the companies that are at least 20 years old.
dfCompany = spark.sql("Select * from companyinfo where ('2022'-EstablishedYear) >20")
dfCompany.show()

+---------+------------------+--------------------+--------------------+--------------+------------+---------------+
|CompanyID|       CompanyName|     CompanyLocation|      CompanyAddress|CompanyContact|ProfitMargin|EstablishedYear|
+---------+------------------+--------------------+--------------------+--------------+------------+---------------+
|    C1003|          Lee Auto|          California|   87985 Linden Pass|          null|  4422985.26|           1994|
|    C1004|       Zoom Dynamo|               Texas| 56 La Follette Pass|          null|  3102258.84|           1993|
|    C1007|       Raido Bikes|              Hawaii|    3925 Clove Drive|          null|  2856030.07|           2001|
|    C1011|      Steel Wheels|             Arizona|83 Merchant Junction|          null|  2025983.74|           1992|
|    C1013|      Craiko Roads|                Iowa|   5 Bowman Junction|          null|  4883173.55|           1990|
|    C1014|        Reo Bailio|          California|      3 Chino

In [75]:
#Fetch the name of the company which has the highest profit margin among those who have ordered Ignition Switch.
df_HighProfit = spark.sql("select companyinfo.companyname,max(ProfitMargin) as ProfitMargin from companyinfo \
                          join companyorders on \
                          companyinfo.CompanyID = companyorders.CompanyID \
                          where ComponentToBeManufactured ='Ignition Switch'\
                          group by companyinfo.companyname \
                          order by ProfitMargin desc Limit(1)")
df_HighProfit.show()

+-----------+------------+
|companyname|ProfitMargin|
+-----------+------------+
| Kioto Moto|  4834622.33|
+-----------+------------+



In [80]:
#Find the total number of companies that have ordered Fender.
dfCompanyOrders.select(dfCompanyOrders['CompanyName']) \
               .where(dfCompanyOrders['ComponentToBeManufactured'] == 'Fender') \
               .count()

40

In [89]:
#total number of exhaust pipes ordered from North Carolina 
dfCompanyOrders.select(dfCompanyOrders['Quantity'])\
               .where((dfCompanyOrders['ComponentToBeManufactured'] =='Exhaust Pipes') &\
                      (dfCompanyOrders['CompanyLocation'] == 'North Carolina'))\
               .show()

+--------+
|Quantity|
+--------+
|   42112|
+--------+



In [91]:
dfExp = spark.sql("select sum(Quantity) from companyorders \
                    where ComponentToBeManufactured = 'Exhaust Pipes' \
                    and CompanyLocation = 'North Carolina'").show()
dfExp.show()

+-------------+
|sum(Quantity)|
+-------------+
|        42112|
+-------------+



In [94]:
# Find the total count of all the orders that has been completed but not delivered yet.

dfCompanyOrders.where((dfCompanyOrders['CompletionStatus'] =='Completed') &\
                      (dfCompanyOrders['DeliveryStatus'] == 'Pending'))\
               .count()


246

In [109]:
import pyspark.sql.functions as F
#Display all the orders whose status is still pending. Reduce the total cost of all such orders by 10%. 
#Store them in a file called DiscountOrders.csv
df_Discount = dfCompanyOrders.where((dfCompanyOrders['CompletionStatus'] == 'Pending'))
df_Discount = df_Discount.withColumn('EstimatedCost',F.round(dfCompanyOrders['EstimatedCost']*10/100,2))
df_Discount.write.csv("/content/DiscountOrders.csv",header=True)
df_Discount.show()

+-------+---------+------------------+---------------+-------------------------+--------+-------------+---------+---------+----------------+--------------+
|OrderID|CompanyID|       CompanyName|CompanyLocation|ComponentToBeManufactured|Quantity|EstimatedCost|OrderDate|  DueDate|CompletionStatus|DeliveryStatus|
+-------+---------+------------------+---------------+-------------------------+--------+-------------+---------+---------+----------------+--------------+
|OD10001|    C1108|       Raido Bikes|        Georgia|             Spring Forks|   21103|    948786.75| 3-Nov-19|24-Dec-19|         Pending|       Pending|
|OD10002|    C1035|   Ilmio Jane Moto| North Carolina|                   Fender|   26272|    489936.93|18-Dec-19|14-Mar-20|         Pending|       Pending|
|OD10005|    C1093|  Quakesons Motors|     California|            Exhaust Pipes|   36773|    610182.96| 8-Dec-19| 9-Mar-20|         Pending|       Pending|
|OD10006|    C1166|         Rigmarole|       New York|          

In [132]:
#Based on the previous question, fetch the total cost to be paid by all the franchises having company name as Laser Wheels
from pyspark.sql.functions import sum
df_Discount.where(df_Discount['CompanyName'] == 'Laser Wheels')\
           .select(sum(df_Discount['EstimatedCost']))\
           .show()
           

+------------------+
|sum(EstimatedCost)|
+------------------+
|        8548640.87|
+------------------+



In [133]:
spark.stop

<bound method SparkSession.stop of <pyspark.sql.session.SparkSession object at 0x7f23d4336410>>